In [1]:
import azureml
import json

from azureml.core.model import Model, InferenceConfig
from azureml.core import Workspace, Datastore, Experiment
from azureml.core.dataset import Dataset

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.45.0


In [ ]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="d6233897-5c9f-47f9-8507-6d4ada2d5183")
resource_group = os.getenv("RESOURCE_GROUP", default="RG_Jaganlal")
workspace_name = os.getenv("WORKSPACE_NAME", default="beanleaf_classifier")
workspace_region = os.getenv("WORKSPACE_REGION", default="centralus")
cluster_name = os.getenv("CLUSTER_NAME", default="beanleaf-classifier-cluster")
dataset_path_in_datastore = os.getenv("BLOB_STORAGE_PATH", default="beanleaf")

In [ ]:
ws = Workspace.from_config()
ws

In [ ]:
from azureml.core import ComputeTarget
target = ComputeTarget(ws, cluster_name)
target

# Connect to Workspace

In [ ]:
# Connect to compute for training
compute_target = ComputeTarget(workspace=ws, name=cluster_name)
print("Compute Target:", compute_target.name)

# Connect to the datastore for the training images
ds = Datastore.get_default(ws)
print("Datastore:", ds.name)

# Connect to the experiment
exp = Experiment(workspace=ws, name='BeanClassifierExperiment')
print("Experiment:", exp.name)

In [ ]:
from azureml.data.datapath import DataPath

# Retrieve default datastore that's automatically created when a workspace is setup
default_blob_store = ws.get_default_datastore()
print("Default datastore's name: {}".format(default_blob_store.name))
data_path = DataPath(datastore=default_blob_store, path_on_datastore='cv_datapath')
print('Data path:', data_path._name)

# Download Beanleaf dataset

In [ ]:
!wget https://storage.googleapis.com/ibeans/train.zip
!wget https://storage.googleapis.com/ibeans/validation.zip
!wget https://storage.googleapis.com/ibeans/test.zip

In [ ]:
!mkdir beans

In [ ]:
!unzip train.zip -d beans/
!unzip test.zip -d beans/
!unzip validation.zip -d beans/

In [ ]:
from IPython.display import Image, display

display(Image('beans/train/healthy/healthy_train.0.jpg'))

# Upload Dataset

### From local directory

In [ ]:
# beanleaf_ds = Dataset.File.upload_directory(src_dir='/Users/jaganlalthoppe/workspace/mlops/azure/mlops-cv-model/data',
#            target=DataPath(ds, data_path._name),
#            show_progress=True)
# beanleaf_ds

dataset_path_in_datastore = "beanleaf"
beanleaf_ds = default_blob_store.upload(
    src_dir = './beans',
    target_path = dataset_path_in_datastore,
    overwrite = True
)

In [ ]:
data_ref = default_blob_store.path(dataset_path_in_datastore).as_mount()

data_ref.path()

In [ ]:
# retrieve an existing datastore in the workspace by name
datastore = Datastore.get(ws, default_blob_store.name)
datastore

In [ ]:
ds